In [1]:
import numpy as np
import tvm
from tvm import te

In [2]:
n = te.var('n')
m = te.var('m')
A = te.placeholder((n, m), name='a')
# (m, n) is the ouput shape 
B = te.compute((m, n), lambda i, j: A[j, i], 'b')
s = te.create_schedule(B.op)
tvm.lower(s, [A, B], simple_mode=True)

#[version = "0.0.5"]
@main = primfn(a_1: handle, b_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [(stride: int32*n: int32)], [], type="auto"),
             b: Buffer(b_2: Pointer(float32), float32, [(stride_1: int32*m: int32)], [], type="auto")}
  buffer_map = {a_1: a, b_1: b}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [n, m], [stride, stride_2: int32], type="auto"), b_1: b_3: Buffer(b_2, float32, [m, n], [stride_1, stride_3: int32], type="auto")} {
  for (i: int32, 0, m) {
    for (j: int32, 0, n) {
      b[((i*stride_1) + (j*stride_3))] = a[((j*stride) + (i*stride_2))]
    }
  }
}

#[metadata]
{
  "root": 1, 
  "nodes": [
    {
      "type_key": ""
    }, 
    {
      "type_key": "Map", 
      "keys": [
        "IntImm"
      ], 
      "data": [2]
    }, 
    {
      "type_key": "Array", 
      "data": [3, 4]
    }, 
    {
      "type_key": "IntImm", 
  

In [3]:
a = np.arange(12, dtype='float32').reshape((3, 4))
b = np.empty((4, 3), dtype='float32')
a, b = tvm.nd.array(a), tvm.nd.array(b)

mod = tvm.build(s, [A, B])
mod(a, b)
print(a)
print(b)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
[[ 0.  4.  8.]
 [ 1.  5.  9.]
 [ 2.  6. 10.]
 [ 3.  7. 11.]]


In [4]:
B = te.compute((m*n, ), lambda i: A[i//m, i%m], name='b')
s = te.create_schedule(B.op)
tvm.lower(s, [A, B], simple_mode=True)

#[version = "0.0.5"]
@main = primfn(a_1: handle, b_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [(stride: int32*n: int32)], [], type="auto"),
             b: Buffer(b_2: Pointer(float32), float32, [(m: int32*n)], [])}
  buffer_map = {a_1: a, b_1: b}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [n, m], [stride, stride_1: int32], type="auto"), b_1: b_3: Buffer(b_2, float32, [(m*n)], [])} {
  for (i: int32, 0, (m*n)) {
    b[i] = a[((floordiv(i, m)*stride) + (floormod(i, m)*stride_1))]
  }
}

#[metadata]
{
  "root": 1, 
  "nodes": [
    {
      "type_key": ""
    }, 
    {
      "type_key": "Map", 
      "keys": [
        "IntImm"
      ], 
      "data": [2]
    }, 
    {
      "type_key": "Array", 
      "data": [3, 4]
    }, 
    {
      "type_key": "IntImm", 
      "attrs": {
        "dtype": "bool", 
        "span": "0", 
        "value": "1"
      }
    }

In [5]:
p, q = te.var('p'), te.var('q')
B = te.compute((p, q), lambda i, j: A[(i*q+j)//m, (i*q+j)%m], name='b')
s = te.create_schedule(B.op)
tvm.lower(s, [A, B], simple_mode=True)

#[version = "0.0.5"]
@main = primfn(a_1: handle, b_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [(stride: int32*n: int32)], [], type="auto"),
             b: Buffer(b_2: Pointer(float32), float32, [(stride_1: int32*p: int32)], [], type="auto")}
  buffer_map = {a_1: a, b_1: b}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [n, m: int32], [stride, stride_2: int32], type="auto"), b_1: b_3: Buffer(b_2, float32, [p, q: int32], [stride_1, stride_3: int32], type="auto")} {
  for (i: int32, 0, p) {
    for (j: int32, 0, q) {
      b[((i*stride_1) + (j*stride_3))] = a[((floordiv(((i*q) + j), m)*stride) + (floormod(((i*q) + j), m)*stride_2))]
    }
  }
}

#[metadata]
{
  "root": 1, 
  "nodes": [
    {
      "type_key": ""
    }, 
    {
      "type_key": "Map", 
      "keys": [
        "IntImm"
      ], 
      "data": [2]
    }, 
    {
      "type_key": "Array", 
      

In [6]:
mod = tvm.build(s, [A, B])
a = np.arange(12, dtype='float32').reshape((3, 4))
b = np.zeros((5, 4), dtype='float32')
a, b = tvm.nd.array(a), tvm.nd.array(b)

mod(a, b)
print(b)

[[ 0.0000000e+00  1.0000000e+00  2.0000000e+00  3.0000000e+00]
 [ 4.0000000e+00  5.0000000e+00  6.0000000e+00  7.0000000e+00]
 [ 8.0000000e+00  9.0000000e+00  1.0000000e+01  1.1000000e+01]
 [-6.8298542e-22  3.0780922e-41  9.1084400e-44  0.0000000e+00]
 [ 8.4077908e-45  1.4012985e-45  1.9669246e+20  4.5832269e-41]]


In [8]:
bi, bj, si, sj = [te.var(name) for name in ['bi', 'bj', 'si', 'sj']]
B = te.compute(((n-bi)//si, (m-bj)//sj), lambda i, j: A[i*si+bi, j*sj+bj], name='b')
s = te.create_schedule(B.op)
mod = tvm.build(s, [A, B, bi, si, bj, sj])

In [9]:
b = tvm.nd.array(np.empty((1, 3), dtype='float32'))
mod(a, b, 1, 2, 1, 1)
np.testing.assert_equal(b.asnumpy(), a.asnumpy()[1::2, 1::1])

In [10]:
b = tvm.nd.array(np.empty((1, 2), dtype='float32'))
mod(a, b, 2, 1, 0, 2)
np.testing.assert_equal(b.asnumpy(), a.asnumpy()[2::1, 0::2])